# Out of core dataframes for Python
## PyParis 2018

In [ ]:
import warnings
warnings.simplefilter('ignore')
import vaex
import vaex_arrow.dataset
import numpy as np
import matplotlib.pylab as plt
np.warnings.filterwarnings('ignore')
%matplotlib inline

# Step 0: reading in data
vaex reads 'anything':
 * `df = vaex.open('super_fast.arrow')`
 * `df = vaex.open('super_fast.hdf5')`
 * `df = vaex.from_pandas(df)`
 * `df = vaex.from_ascii('takes_hours.asc')`
 * `df = vaex.from_csv('this_may_be_slow.csv')`
 * `df = vaex.from_arrays(x=x, y=y)`

In [ ]:
!ls -lh /Users/maartenbreddels/datasets/nytaxi/nyc_taxi2015.arrow

In [ ]:
%%time
df = vaex.open("/Users/maartenbreddels/datasets/nytaxi/nyc_taxi2015.arrow")

In [ ]:
df

## Pandas like, but uses expressions

In [ ]:
df.trip_distance.values

In [ ]:
bytes_per_column = df.trip_distance.values.dtype.itemsize * len(df)
f"{bytes_per_column:,}"

In [ ]:
df.trip_distance

In [ ]:
np.log10(df.trip_distance)

# Expressions / Virtual columns
![expres](./meme-expressions.jpg)

In [ ]:
df.mean(df.tip_amount)

In [ ]:
df.tip_amount/df.total_amount

In [ ]:
df.mean(df.tip_amount/df.total_amount)

In [ ]:
df['tip_percentage'] = df.tip_amount/df.total_amount

In [ ]:
df.info()

In [ ]:
df.mean(df.tip_percentage)

# Lazy is good
See item 10/11 of ["10 Things I Hate About pandas"](http://wesmckinney.com/blog/apache-arrow-pandas-internals/) by Wes McKinney
> When you write df[df.c < 0].d.sum(), pandas creates a temporary DataFrame df[df.c < 0] then sums the d column of that temporary object. If df contains a lot of columns, this is ridiculously wasteful.

In [ ]:
# this would cost 46+GB RAM using Pandas
# and wouldn't be possible on this laptop
print(df[df.tip_amount < 10].tip_amount.mean())

# (Binned) Statistics
Strong focus on statistics (on regular grids)
## 0 dimensional

In [ ]:
df.count()

In [ ]:
df.count(df.pickup_latitude)

In [ ]:
df.mean(df.pickup_latitude)

## 1 dimensional

In [ ]:
df.count(binby=df.pickup_latitude, limits=[40.5, 41], shape=100)

In [ ]:
plt.plot(_)

## 2 dimensional

In [ ]:
counts2d = df.count(binby=[df.pickup_longitude, df.pickup_latitude], shape=128)
print(counts2d.shape)

In [ ]:
plt.imshow(np.log10(counts2d+1).T, origin='lower')

In [ ]:
limits = df.limits([df.pickup_longitude, df.pickup_latitude], "98%")
limits

In [ ]:
%%time
df.plot(df.pickup_longitude, df.pickup_latitude, f="log1p",
        limits=limits, figsize=(10,8), shape=512, colormap="viridis")

In [ ]:
df.plot(df.pickup_longitude, df.pickup_latitude,
        z='trip_distance:0,30,3',
        what=[vaex.stat.count(), vaex.stat.mean(df.tip_amount)],
        visual=dict(column="z", row='what'),
        f="log1p",
        limits=limits, figsize=(16,10), shape=512, colormap="viridis")

## Where to pick up customers?

In [ ]:
df.plot(df.pickup_longitude, df.pickup_latitude, what=vaex.stat.mean(df.total_amount),
        vmin=0, vmax=50, shape=512, figsize=(10,8), limits=limits, colormap="Greys")

In [ ]:
df.trip_distance.minmax()

In [ ]:
df.plot1d(df.trip_distance, limits=[0, 50])

In [ ]:
# no memory copy! and not wasting 46 GB of memory
df_clean = df[(df.trip_distance > 0) & (df.trip_distance < 40)]

In [ ]:
df_clean.total_amount/df_clean.trip_distance

In [ ]:
df_clean['ratio'] = df_clean.total_amount / df_clean.trip_distance
df_clean.ratio

In [ ]:
df_clean.plot(df_clean.pickup_longitude, df_clean.pickup_latitude,
         what=vaex.stat.mean(df_clean.ratio),
         vmin=0, vmax=15,
         shape=512, figsize=(10,8), limits=limits, colormap="Greys")

# Interactive

In [ ]:
import vaex
import pylab as plt
df = vaex.open("/Users/maartenbreddels/datasets/nytaxi/nyc_taxi2015.arrow")
limits = df.limits([df.pickup_longitude, df.pickup_latitude], "98%")

In [ ]:
w = df.plot_widget(df.pickup_longitude, df.pickup_latitude, selection=[None, 'JFK', 'LaG'],
             controls_selection=True, f='log',
             shape=512, figsize=(10,8), limits=limits, colormap="afmhot")#, backend='ipyleaflet')

In [ ]:
limits = [1, 20]
shape = 30
plt.figure(figsize=(10,7))
df.plot1d(df.tip_amount, selection='LaG', n=True, limits=limits, shape=shape, lw=3)
df.plot1d(df.tip_amount, selection='JFK', n=True, limits=limits, shape=shape, lw=3)
plt.legend(fontsize=20)

# Heavy calculations

In [ ]:
def arc_distance(theta_1, phi_1, theta_2, phi_2):
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    distance = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return distance * 6400

In [ ]:
# distance Groningen - Paris (in kilometer)
arc_distance(53.1739086, 6.5990374, 48.8588376,2.2768492)

In [ ]:
arc_distance_expression = arc_distance(df.pickup_longitude,
                                       df.pickup_latitude,
                                       df.dropoff_longitude,
                                       df.dropoff_latitude)

In [ ]:
arc_distance_expression

In [ ]:
df['arc_distance'] = arc_distance(df.pickup_longitude,
                                  df.pickup_latitude,
                                  df.dropoff_longitude,
                                  df.dropoff_latitude)

In [ ]:
df

In [ ]:
%%time
df.sum(df.total_amount)

In [ ]:
%%time
df.sum(df.arc_distance)

In [ ]:
df['arc_distance_jit'] = df.arc_distance.jit_numba()
# ds['arc_distance_jit'] = ds.arc_distance.jit_pythran()

In [ ]:
%%time
df.sum(df.arc_distance_jit)

In [ ]:
# if you got RAM to waste (I don't on this machine)
# TODO: optimize materialize
# ds.materialize(ds.arc_distance, inplace=True)

In [ ]:
# %%timeit -n1 -r3
# ds.sum(ds.arc_distance)

# Expression advantages
  * No information loss
  * Optimization (jit: numba/pythran)
  * Derivatives (error/uncertainty propagation)
  * No memory waste, only calculate what is needed  
  * Do 'remote' calculations


# Remote datasets

In [ ]:
import vaex
import numpy as np

In [ ]:
df_remote = vaex.open('ws://gaia:10000/nyc_taxi_all')
# ds_remote = vaex.open('ws://localhost:9000/nyc_taxi_all')
df_remote

In [ ]:
df_remote.total_amount / 100

In [ ]:
def arc_distance(theta_1, phi_1, theta_2, phi_2):
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    distance = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return distance * 6400

In [ ]:
df_remote['arc_distance'] = arc_distance(df_remote.pickup_longitude,
                                         df_remote.pickup_latitude,
                                         df_remote.dropoff_longitude,
                                         df_remote.dropoff_latitude)

In [ ]:
df_remote.arc_distance

In [ ]:
df_remote['arc_distance_jit'] = df_remote.arc_distance.jit_numba()

In [ ]:
df_remote.arc_distance_jit

In [ ]:
%%time
df_remote.mean('arc_distance', progress=True)

In [ ]:
%%time
df_remote.mean('arc_distance_jit', progress=True)

In [ ]:
limits = df_remote.limits([df_remote.pickup_longitude, df_remote.pickup_latitude], "98%")

In [ ]:
# ds_remote.plot_widget(ds_remote.pickup_longitude, ds_remote.pickup_latitude, backend='ipyleaflet', f='log', shape=400)

In [ ]:
df_remote.plot_widget(df_remote.pickup_longitude, df_remote.pickup_latitude,
                     selection=[None, 'JFK', 'LaG'],
                     controls_selection=True, f='log',
                     shape=512, figsize=(10,8), limits=limits, colormap="afmhot")

In [ ]:
import matplotlib.pylab as plt

In [ ]:
limits = [1, 20]
shape = 30
plt.figure(figsize=(10,7))
df_remote.plot1d(df_remote.tip_amount, selection='LaG', n=True, limits=limits, shape=shape, lw=3)
df_remote.plot1d(df_remote.tip_amount, selection='JFK', n=True, limits=limits, shape=shape, lw=3)
plt.legend(fontsize=20)